https://stepik.org/lesson/1576188/step/6

In [ ]:
# @title Расчёт размера изображения после применения слоёв Conv2d и MaxPool2d

H_in, W_in = 32, 32 # высота и ширина входного изображения

# 1-й слой (Conv2d)
p = (2, 2)          # padding, количество пикселей, добавленных к границам изображения
k = (5, 5)          # kernel_size, размер ядра свертки
s = (1, 1)          # stride, шаг с которым ядро перемещается по изображению
d = (1, 1)          # dilation, расстояние между элементами ядра (когда не все пиксели учитываем)

H_1 = (H_in + 2 * p[0] - d[0] * (k[0] - 1) - 1) / s[0] + 1
W_1 = (H_in + 2 * p[1] - d[1] * (k[1] - 1) - 1) / s[1] + 1

# 2-й слой (MaxPool2d)
H_2 = H_1 // 2
W_2 = W_1 // 2

# 3-й слой (Conv2d)
p = (1, 1)
k = (3, 3)
s = (1, 1)
d = (1, 1)

H_3 = (H_2 + 2 * p[0] - d[0] * (k[0] - 1) - 1) / s[0] + 1
W_3 = (W_2 + 2 * p[1] - d[1] * (k[1] - 1) - 1) / s[1] + 1

# 4-й слой (MaxPool2d)
H_4 = H_3 // 2
W_4 = W_3 // 2

print(H_4, W_4)

8.0 8.0


In [ ]:
import torch
import torch.nn as nn

class ImageNormalize():
    def __call__(self, tensor: torch.Tensor) -> torch.Tensor:
        min_t = tensor.min()
        max_t = tensor.max()
        return (tensor - min_t) / (max_t - min_t)

# генерация образов выборки
total = 100     # размер выборки
H, W = 32, 32   # размер изображений
circle = torch.tensor([[0, 0, 0, 255, 255, 255, 255, 0, 0, 0],
                       [0, 255, 255, 255, 255, 255, 255, 255, 255, 0],
                       [0, 255, 255, 255, 255, 255, 255, 255, 255, 0],
                       [255, 255, 255, 255, 255, 255, 255, 255, 255, 255],
                       [255, 255, 255, 255, 255, 255, 255, 255, 255, 255],
                       [255, 255, 255, 255, 255, 255, 255, 255, 255, 255],
                       [255, 255, 255, 255, 255, 255, 255, 255, 255, 255],
                       [0, 255, 255, 255, 255, 255, 255, 255, 255, 0],
                       [0, 255, 255, 255, 255, 255, 255, 255, 255, 0],
                       [0, 0, 0, 255, 255, 255, 255, 0, 0, 0]], dtype=torch.float32)
Hc, Wc = circle.size()


def _generate_img(_H, _W, _Hc, _Wc, _x, _y, _circle, _tr): # вспомогательная функция
    img = torch.rand(_H, _W) * 20
    img[_x:_x+_Hc, _y:_y+Wc] = _circle
    return _tr(img.view(1, 1, _H, _W))


# Функция для создания объекта нормализатора с применением (через call)
transform = ImageNormalize()

data_y = torch.tensor([(torch.randint(0, H-Hc, (1, )), torch.randint(0, W-Wc, (1, ))) for _ in range(total)])
data_x = torch.cat([_generate_img(H, W, Hc, Wc, _x[0], _x[1], circle, transform) for _x in data_y], dim=0)

torch.manual_seed(1)

model = nn.Sequential(
    nn.Conv2d(1, 16, (5, 5), stride=1, padding=2, bias=True),
    nn.ReLU(inplace=True),
    nn.MaxPool2d((2, 2), stride=2),
    nn.Conv2d(16, 32, (3, 3), stride=1, padding=1, bias=True),
    nn.ReLU(inplace=True),
    nn.MaxPool2d((2, 2), stride=2),
    nn.Flatten(),
    nn.Linear(32 * 8 * 8, 2, bias=True)  # 2 класса
)

model.eval()

predict = model(data_x)

loss_func = nn.MSELoss()

Q = loss_func(predict, data_y.type(torch.float32)).item()

# print(Q)

In [ ]:
# @title Расчёт размера изображения после применения слоя torch.nn.Conv2d
H_in, W_in = 8, 8   # высота и ширина входного изображения
p = (1, 1)          # padding, количество пикселей, добавленных к границам изображения
k = (3, 3)          # kernel_size, размер ядра свертки
s = (1, 1)          # stride, шаг с которым ядро перемещается по изображению
d = (1, 1)          # dilation, расстояние между элементами ядра (когда не все пиксели учитываем)

H_out = (H_in + 2 * p[0] - d[0] * (k[0] - 1) - 1) / s[0] + 1
W_out = (H_in + 2 * p[1] - d[1] * (k[1] - 1) - 1) / s[1] + 1

H_out, W_out

(8.0, 8.0)

Размер изображения после применения слоя `torch.nn.Conv2d` можно рассчитать по следующей формуле:

$$
H_{\text{out}} = \left\lfloor \frac{H_{\text{in}} + 2 \times padding[0] - dilation[0] \times (kernel\_size[0] - 1) - 1}{stride[0]} + 1 \right\rfloor
$$

$$
W_{\text{out}} = \left\lfloor \frac{W_{\text{in}} + 2 \times padding[1] - dilation[1] \times (kernel\_size[1] - 1) - 1}{stride[1]} + 1 \right\rfloor
$$

Где:

- $H_{\text{in}}$$ и $$W_{\text{in}}$ — высота и ширина входного изображения[4].
- $\text{padding}$ — количество пикселей, добавленных к границам изображения[1][4].
- $\text{kernel_size}$ — размер ядра свертки[1][4].
- $\text{stride}$ — шаг, с которым ядро перемещается по изображению[1][4].
- $\text{dilation}$ — расстояние между элементами ядра[4].

**Пример:**
Если входное изображение имеет размер $28 \times 28$, а параметры слоя заданы как `Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=0)`, то размер выходного изображения будет:

$$
H_{\text{out}} = \left\lfloor \frac{28 + 2 \times 0 - 1 \times (5 - 1) - 1}{1} + 1 \right\rfloor = 24
$$

$$
W_{\text{out}} = \left\lfloor \frac{28 + 2 \times 0 - 1 \times (5 - 1) - 1}{1} + 1 \right\rfloor = 24
$$

Таким образом, выходное изображение будет иметь размер $24 \times 24$[2][4].

Citations:
[1] https://proproprogs.ru/nn_pytorch/pytorch-klassy-conv2d-i-maxpool2d

[2] https://discuss.pytorch.org/t/pytorch-nn-conv2d-computation-of-number-of-features-output-from-nn-conv2d/150750

[3] https://www.bizkit.ru/2020/02/25/17169/

[4] https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html


[5] https://ru.stackoverflow.com/questions/1170799/k%D0%B0%D0%BA-%D0%BF%D0%BE%D0%BC%D0%B5%D0%BD%D1%8F%D0%B5%D1%82%D1%81%D1%8F-%D1%80%D0%B0%D0%B7%D0%BC%D0%B5%D1%80-%D0%B8%D0%B7%D0%BE%D0%B1%D1%80%D0%B0%D0%B6%D0%B5%D0%BD%D0%B8%D1%8F-%D0%BF%D0%BE-%D1%81%D0%B2%D1%91%D1%80%D1%82%D0%BE%D1%87%D0%BD%D0%BE%D0%B9-%D1%81%D0%B5%D1%82%D0%B8

[6] https://stepik.org/lesson/309343/step/5

[7] https://habr.com/ru/articles/456740/

[8] https://stackoverflow.com/questions/56675943/meaning-of-parameters-in-torch-nn-conv2d

[9] https://robocraft.ru/computervision/427

[10] https://habr.com/ru/articles/454986/

---
Answer from Perplexity: pplx.ai/share